In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df = pd.read_json('/kaggle/input/business-json/task_2_raw.json').transpose()
df.reset_index(inplace=True)
df.rename(columns={'index': 'QID'}, inplace=True)
print(df)

     QID                                   question  \
0      0  Are these established in the same decade?   
1      1  Are these established in the same decade?   
2      2  Are these established in the same decade?   
3      3  Are these established in the same decade?   
4      4  Are these established in the same decade?   
..   ...                                        ...   
495  495               Are these greek restaurants?   
496  496               Are these greek restaurants?   
497  497               Are these greek restaurants?   
498  498               Are these greek restaurants?   
499  499               Are these greek restaurants?   

                                          sentence_1  \
0      Banco do Brasil has headquarters in Brasília.   
1         Banco do Brasil has origin country Brazil.   
2       Banco do Brasil has CEO John VI of Portugal.   
3    State Bank of India has headquarters in Mumbai.   
4      Banco do Brasil has headquarters in Brasília.   
.. 

In [3]:


entity_df = pd.read_csv("/kaggle/input/business-json/Knowledge.csv")
# Merge the original DataFrame with the entity DataFrame
# Expand the entities and entity_titles columns to match each entity with its title
merged_df = df.explode(['entities', 'entity_titles'])

# Merge with entity_df on entity_titles and entity_name
merged_df = merged_df.merge(entity_df, left_on='entity_titles', right_on='Entity_name', how='left')

# Replace the original 'entities' with the correct 'entity_id' from entity_df
merged_df['entities'] = merged_df['Entity_ID']

# Group back to the original structure
updated_df = merged_df.groupby(['QID', 'question', 'sentence_1', 'sentence_2', 'answer'], as_index=False).agg({
    'entities': lambda x: list(x),
    'entity_titles': lambda x: list(x)
})

print(updated_df)


     QID                                   question  \
0      0  Are these established in the same decade?   
1      1  Are these established in the same decade?   
2      2  Are these established in the same decade?   
3      3  Are these established in the same decade?   
4      4  Are these established in the same decade?   
..   ...                                        ...   
495  495               Are these greek restaurants?   
496  496               Are these greek restaurants?   
497  497               Are these greek restaurants?   
498  498               Are these greek restaurants?   
499  499               Are these greek restaurants?   

                                          sentence_1  \
0      Banco do Brasil has headquarters in Brasília.   
1         Banco do Brasil has origin country Brazil.   
2       Banco do Brasil has CEO John VI of Portugal.   
3    State Bank of India has headquarters in Mumbai.   
4      Banco do Brasil has headquarters in Brasília.   
.. 

In [4]:
print(500 - df.isna().sum())
print((500 - df.isna().sum()).sum())

QID              500
question         500
sentence_1       500
sentence_2       500
answer           500
entities         500
entity_titles    500
dtype: int64
3500


In [5]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
import os
warnings.filterwarnings("ignore")
import time
from tqdm import tqdm
import torch
import ast

In [6]:
columns = ['QID', 'question', 'sentence_1', 'sentence_2', 'answer', 'entities',
       'entity_titles']

In [7]:
for column in columns:
    print(df[column].value_counts())

QID
499    1
0      1
1      1
2      1
3      1
      ..
23     1
24     1
25     1
26     1
27     1
Name: count, Length: 500, dtype: int64
question
Do they have the headquarters in the same city?    300
Are these established in the same decade?          160
Are these japanese restaurants?                     10
Are these british restaurants?                      10
Are these american restaurants?                     10
Are these swedish restaurants?                       5
Are these greek restaurants?                         5
Name: count, dtype: int64
sentence_1
Hotto Motto is a type of Restaurant.                    8
Whataburger is a type of Restaurant.                    4
Bankia has origin country Spain.                        3
KeyBank has headquarters in Cleveland.                  3
Sumitomo Mitsui Trust Bank has origin country Japan.    3
                                                       ..
Porsche has origin country Germany.                     1
Mercedes-Benz has Pau

In [8]:
!apt-get install espeak-ng -y




The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 39 not upgraded.
Need to get 4526 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-10ubuntu0.1 [3956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libespeak-ng1 amd64 1.50+dfsg-10ubuntu0.1 [207 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 espeak-ng amd64 1.50+dfsg-10ubuntu0.1 [343 kB]
Fetched 4526 kB in 1s (3645 kB/s)
Selecting previously unselected package lib

In [9]:
!git clone https://github.com/coqui-ai/TTS
%cd TTS

Cloning into 'TTS'...
remote: Enumerating objects: 32844, done.
remote: Counting objects: 100% (3521/3521), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 32844 (delta 3433), reused 3395 (delta 3395), pack-reused 29323 (from 1)
Receiving objects: 100% (32844/32844), 166.12 MiB | 34.56 MiB/s, done.
Resolving deltas: 100% (23858/23858), done.
/kaggle/working/TTS


In [10]:
!pip install -e .

Obtaining file:///kaggle/working/TTS
  Installing build dependencies ... - \ | / - \ | / - \ | / - done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... - \ done
  Preparing editable metadata (pyproject.toml) ... - done
  Using cached packaging-24.1-py3-none-any.whl.metadata (3.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 19.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 62.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 44.2 MB/s eta 0:00:00

In [11]:
from TTS.api import TTS
device = "cuda" if torch.cuda.is_available() else "cpu"
print(TTS().list_models())
print(device)

cuda


In [12]:
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False).to(device)

 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--tacotron2-DDC
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--hifigan_v2
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linea

In [13]:
%cd /kaggle/working

/kaggle/working


In [14]:
audio_dir = 'Task_2_aud_files'
os.makedirs(audio_dir, exist_ok=True)

df_data = []

for i, entry in tqdm(updated_df.iterrows(), total=len(updated_df), desc="Processing Sentences"):
    q_id = int(entry[0])
    question = str(entry[1])
    sen_1 = str(entry[2])
    sen_2 = str(entry[3])
    ans = str(entry[4])
    sen_1_filename = f'{audio_dir}/Sentence_{q_id}_1.wav'
    sen_2_filename = f'{audio_dir}/Sentence_{q_id}_2.wav'
    ent = str(entry[5])
    ent_titles = (entry[6])
    
    
    df_data.append(
    {
'QID' : q_id,
'question' :  question,     
'sentence_1' : sen_1,
'sentence_2' : sen_2,
'sentence_1_filename' : ans,
'sentence_2_filename' : sen_1_filename,
'answer' : sen_2_filename,            
'entities' : ent,                            
'entity_titles' : ent_titles,
    }
    )
    
#     print(entry)
    try:
        tts.tts_to_file(text=sen_1, file_path=sen_1_filename)
    except Exception as e:
        print(f"Error processing sentence 1 of q_id {q_id}: {e}")
        continue    
    
    try:
        tts.tts_to_file(text=sen_2, file_path=sen_2_filename)
    except Exception as e:
        print(f"Error processing sentence 2 of q_id {q_id}: {e}")
        continue
        

#     try:
#         tts.tts_to_file(text=text_up, file_path=wav_filename)
#     except Exception as e:
#         print(f"Error processing entity {link}: {e}")
#         continue
    
#     print(entry)
#     time.sleep(0.5)
#     print(text_up)
#     wav_filename = f'entity_aud_files/Entity_{link}.wav'
    
#     try:
#         tts.tts_to_file(text=text_up, file_path=wav_filename)
#     except Exception as e:
#         print(f"Error processing entity {link}: {e}")
#         continue
    
#     df_data.append({
#         'ID': link,
#         'entity_name': text_up,
#         'wav_addr': wav_filename
#     })

Task_2_df = pd.DataFrame(df_data)

Processing Sentences:   0%|          | 0/500 [00:00<?, ?it/s]

 > Text splitted to sentences.
['Banco do Brasil has headquarters in Brasília.']
banco do brasil has headquarters in brasília.
 [!] Character 'í' not found in the vocabulary. Discarding it.
 > Processing time: 1.5186972618103027
 > Real-time factor: 0.40742742143903515
 > Text splitted to sentences.
['Banco do Brasil has CEO John VI of Portugal.']


Processing Sentences:   0%|          | 1/500 [00:01<16:31,  1.99s/it]

 > Processing time: 0.42514514923095703
 > Real-time factor: 0.10736726384165525
 > Text splitted to sentences.
['Banco do Brasil has origin country Brazil.']
 > Processing time: 0.4018113613128662
 > Real-time factor: 0.10950094568109427
 > Text splitted to sentences.
['Banco do Brasil has origin country Brazil.']


Processing Sentences:   0%|          | 2/500 [00:02<10:52,  1.31s/it]

...

 > Processing time: 0.5637269020080566
 > Real-time factor: 0.1039590708992176
 > Text splitted to sentences.
['Firehouse Subs is a type of Restaurant.']
 > Processing time: 0.33647990226745605
 > Real-time factor: 0.09855714459348307
 > Text splitted to sentences.
['Hotto Motto is a type of Restaurant.']


Processing Sentences: 100%|█████████▉| 499/500 [10:05<00:01,  1.04s/it]

 > Processing time: 0.30835437774658203
 > Real-time factor: 0.09762253085963896
 > Text splitted to sentences.
["Dunkin' has William Rosenberg as CEO."]
 > Processing time: 0.4252033233642578
 > Real-time factor: 0.10675593550944941
 > Text splitted to sentences.
['Panda Express was established on 1973-01-01']


Processing Sentences: 100%|██████████| 500/500 [10:06<00:00,  1.21s/it]

 > Processing time: 0.5757894515991211
 > Real-time factor: 0.10779919005366646


In [15]:
# QID                                                          0
# question             Are these established in the same decade?
# sentence_1       Banco do Brasil has headquarters in Brasília.
# sentence_2        Banco do Brasil has CEO John VI of Portugal.
# answer                                                     Yes
# entities                                    [Q610817, Q610817]
# entity_titles               [Banco do Brasil, Banco do Brasil]

In [16]:
!zip -r Task_2_aud_files.zip /kaggle/working/Task_2_aud_files

  adding: kaggle/working/Task_2_aud_files/ (stored 0%)
  adding: kaggle/working/Task_2_aud_files/Sentence_407_2.wav (deflated 22%)
  adding: kaggle/working/Task_2_aud_files/Sentence_143_2.wav (deflated 19%)
  adding: kaggle/working/Task_2_aud_files/Sentence_459_1.wav (deflated 27%)
  adding: kaggle/working/Task_2_aud_files/Sentence_135_2.wav (deflated 24%)
  adding: kaggle/working/Task_2_aud_files/Sentence_81_2.wav (deflated 16%)
  adding: kaggle/working/Task_2_aud_files/Sentence_28_2.wav (deflated 22%)
  adding: kaggle/working/Task_2_aud_files/Sentence_235_2.wav (deflated 19%)
  adding: kaggle/working/Task_2_aud_files/Sentence_52_2.wav (deflated 25%)
  adding: kaggle/working/Task_2_aud_files/Sentence_150_1.wav (deflated 25%)
  adding: kaggle/working/Task_2_aud_files/Sentence_2_1.wav (deflated 20%)
  adding: kaggle/working/Task_2_aud_files/Sentence_175_1.wav (deflated 24%)
  adding: kaggle/working/Task_2_aud_files/Sentence_226_1.wav (deflated 21%)
  adding: kaggle/working/Task_2_aud_fi

In [17]:
# from IPython.display import FileLink
# FileLink(r'Task_2_aud_files.zip')